In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import random
import os
import math

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 42
seed_everything(SEED)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {DEVICE}")

# ======================================================
# [V7] 길이별 앙상블 전략 하이퍼파라미터
# ======================================================
BATCH_SIZE = 64
LR_BASE = 1e-3
WARMUP_EPOCHS = 3
EPOCHS_BASE = 40
DROPOUT = 0.2
MAX_SEQ_LEN = 30
GRAD_CLIP = 1.0

# Transformer 파라미터
D_MODEL = 128
NHEAD = 4
NUM_LAYERS = 3

# LSTM 파라미터
HIDDEN_DIM = 256       # LSTM hidden dimension
LSTM_LAYERS = 2        # LSTM 레이어 수
BIDIRECTIONAL = True   # 양방향 LSTM

# 길이별 임계값 (이론에 기반)
SHORT_THRESHOLD = 15   # 짧은 에피소드: <= 15
LONG_THRESHOLD = 40    # 긴 에피소드: > 40

print(f"[V7] 길이별 앙상블 전략")
print(f"  - 짧은 에피소드 (<= {SHORT_THRESHOLD}): LSTM만 사용")
print(f"  - 중간 에피소드 ({SHORT_THRESHOLD} ~ {LONG_THRESHOLD}): LSTM 60% + Transformer 40%")
print(f"  - 긴 에피소드 (> {LONG_THRESHOLD}): Transformer만 사용")

In [13]:
# ======================================================
# 1. 경로 설정 및 Train 로드
# ======================================================
BASE_DIR = "./open_track1" # 데이터가 있는 폴더명 (환경에 맞게 수정)
if not os.path.exists(BASE_DIR): BASE_DIR = "." # 폴더 없으면 현재 경로

TRAIN_PATH = os.path.join(BASE_DIR, "train.csv")
TEST_META_PATH = os.path.join(BASE_DIR, "test.csv") # 사용자님이 올리신 파일명
MATCH_PATH = os.path.join(BASE_DIR, "match_info.csv")

# 1. Train 로드
train_df = pd.read_csv(TRAIN_PATH)
print(f"✅ Train Loaded: {train_df.shape}")

# 2. Test 통합 로드 (핵심 수정 사항)
if os.path.exists(TEST_META_PATH):
    test_meta = pd.read_csv(TEST_META_PATH)
    print(f"ℹ️ Reading {len(test_meta)} test files...")
    
    test_dfs = []
    # tqdm으로 진행상황 확인
    for _, row in tqdm(test_meta.iterrows(), total=len(test_meta), desc="Loading Test CSVs"):
        rel_path = row['path']
        
        # 경로 보정: 1) 그대로, 2) 폴더 붙여서, 3) 파일명만으로 찾기
        paths_to_try = [
            rel_path,
            os.path.join(BASE_DIR, rel_path.lstrip("./")),
            os.path.join(BASE_DIR, "test", str(row['game_id']), os.path.basename(rel_path))
        ]
        
        for p in paths_to_try:
            if os.path.exists(p):
                test_dfs.append(pd.read_csv(p))
                break
        
    if test_dfs:
        test_df = pd.concat(test_dfs, ignore_index=True)
        print(f"✅ Test Data Merged: {test_df.shape}")
    else:
        raise FileNotFoundError("Test 파일들을 찾을 수 없습니다.")
else:
    raise FileNotFoundError("test.csv 파일이 없습니다.")

# 3. Match Info 병합
if os.path.exists(MATCH_PATH):
    match_info = pd.read_csv(MATCH_PATH)
    match_subset = match_info[['game_id', 'home_team_id', 'venue']]
    train_df = pd.merge(train_df, match_subset, on='game_id', how='left')
    test_df = pd.merge(test_df, match_subset, on='game_id', how='left')

# 4. 전처리 (결측치 채우기)
def preprocess(df):
    if 'home_team_id' in df.columns:
        df['is_home'] = (df['team_id'] == df['home_team_id']).astype(float)
    else:
        df['is_home'] = 0.5
        
    # V2 로직 작동을 위해 end_x가 없으면 0으로 채움
    if 'end_x' not in df.columns:
        df['end_x'] = 0.0
        df['end_y'] = 0.0
    else:
        df['end_x'] = df['end_x'].fillna(0.0)
        df['end_y'] = df['end_y'].fillna(0.0)
    return df

train_df = preprocess(train_df)
test_df = preprocess(test_df)

ID_COL = 'game_episode' if 'game_episode' in train_df.columns else 'episode_id'
print(f"Data Ready. ID Column: {ID_COL}")

✅ Train Loaded: (356721, 15)
ℹ️ Reading 2414 test files...


Loading Test CSVs: 100%|██████████| 2414/2414 [00:06<00:00, 367.46it/s]


✅ Test Data Merged: (53110, 15)
Data Ready. ID Column: game_episode


In [14]:
# 1. 범주형 변수 정의 (Train 데이터 분석 기반)
TOP_TYPES = ['Pass', 'Carry', 'Recovery', 'Interception', 'Duel', 'Tackle', 
             'Throw-In', 'Clearance', 'Intervention', 'Block', 'Pass_Freekick', 
             'Cross', 'Goal Kick', 'Error', 'Shot']
ALL_RESULTS = ['Successful', 'Unsuccessful', 'On Target', 'Yellow_Card', 
               'Blocked', 'Keeper Rush-Out', 'Low Quality Shot', 'Off Target']

def make_features(group):
    n = len(group)
    
    # --- 1. 기본 좌표 & Context ---
    sx = group['start_x'].values / 105.0
    sy = group['start_y'].values / 68.0
    ex = group['end_x'].values / 105.0
    ey = group['end_y'].values / 68.0
    is_home = group['is_home'].values
    
    # --- 2. 시간 & 속도 ---
    if 'time_seconds' in group.columns:
        times = group['time_seconds'].values
        dt = np.zeros(n, dtype=np.float32)
        dt[1:] = times[1:] - times[:-1]
        dt = np.maximum(dt, 0.1)
    else:
        dt = np.ones(n, dtype=np.float32)

    # --- 3. 이동량 & 거리 계산 ---
    dx = ex - sx
    dy = ey - sy
    dist_meter = np.sqrt((dx*105)**2 + (dy*68)**2)
    
    # --- 4. 통계량 (Lagged Features) ---
    cumsum_dx = np.cumsum(dx) / 105.0
    cumsum_dy = np.cumsum(dy) / 68.0
    
    lag_dist_m = np.roll(dist_meter, 1); lag_dist_m[0] = 0
    lag_cumsum_dx = np.roll(cumsum_dx, 1); lag_cumsum_dx[0] = 0
    lag_cumsum_dy = np.roll(cumsum_dy, 1); lag_cumsum_dy[0] = 0
    
    lag_dt = np.roll(dt, 1); lag_dt[0] = 1.0
    lag_speed = lag_dist_m / np.maximum(lag_dt, 0.1)
    
    # --- 5. 선수 정보 ---
    if 'player_id' in group.columns:
        p_ids = group['player_id'].values
        is_same = np.zeros(n, dtype=np.float32)
        is_same[1:] = (p_ids[1:] == p_ids[:-1]).astype(np.float32)
    else:
        is_same = np.zeros(n, dtype=np.float32)

    # --- 6. 진행률 & 기타 ---
    progress = np.arange(n) / max(n-1, 1)
    is_second_half = (group['period_id'].values > 1).astype(np.float32) if 'period_id' in group.columns else np.zeros(n)
    
    # ======================================================
    # [V6 Phase 1] 공간 피처 추가
    # ======================================================
    # 6-1. 골대까지 거리 (상대 골대 = 오른쪽)
    GOAL_X = 105.0
    GOAL_Y = 34.0  # 골대 중앙
    
    sx_real = sx * 105.0
    sy_real = sy * 68.0
    dist_to_goal = np.sqrt((sx_real - GOAL_X)**2 + (sy_real - GOAL_Y)**2) / 105.0  # 정규화
    
    # 6-2. 골대 방향 각도 (sin/cos)
    dx_to_goal = GOAL_X - sx_real
    dy_to_goal = GOAL_Y - sy_real
    angle_to_goal = np.arctan2(dy_to_goal, dx_to_goal)
    angle_sin = np.sin(angle_to_goal)
    angle_cos = np.cos(angle_to_goal)
    
    # 6-3. 경계선까지 거리
    dist_to_sideline = np.minimum(sy_real, 68.0 - sy_real) / 68.0  # 위/아래 터치라인
    dist_to_endline = np.minimum(sx_real, 105.0 - sx_real) / 105.0  # 좌/우 골라인
    
    # 6-4. 필드 구역 (3구역: 수비/중앙/공격)
    def get_zone(x_norm):
        if x_norm < 35.0/105.0:
            return 0  # 수비 구역
        elif x_norm < 70.0/105.0:
            return 1  # 중앙 구역
        else:
            return 2  # 공격 구역
    
    zones = np.array([get_zone(x) for x in sx])
    zone_onehot = np.zeros((n, 3), dtype=np.float32)
    for i, z in enumerate(zones):
        zone_onehot[i, z] = 1.0
    
    # --- 7. 범주형 변수 One-hot Encoding ---
    types_onehot = np.zeros((n, len(TOP_TYPES) + 1), dtype=np.float32)
    curr_types = group['type_name'].values
    for i, t in enumerate(curr_types):
        if t in TOP_TYPES:
            idx = TOP_TYPES.index(t)
            types_onehot[i, idx] = 1.0
        else:
            types_onehot[i, -1] = 1.0

    results_onehot = np.zeros((n, len(ALL_RESULTS) + 1), dtype=np.float32)
    curr_results = group['result_name'].values
    for i, r in enumerate(curr_results):
        if r in ALL_RESULTS:
            idx = ALL_RESULTS.index(r)
            results_onehot[i, idx] = 1.0
        else:
            results_onehot[i, -1] = 1.0

    features = []
    
    # --- 8. 시퀀스 결합 ---
    for i in range(n):
        # 스칼라 피처 (11개 → 16개)
        scalars = [
            sx[i], sy[i],                    # 0-1: 위치
            lag_cumsum_dx[i], lag_cumsum_dy[i],  # 2-3: 누적 흐름
            lag_dist_m[i] / 100.0,           # 4: 직전 거리
            lag_speed[i] / 10.0,             # 5: 직전 속도
            dt[i] / 10.0,                    # 6: 시간 경과
            progress[i],                     # 7: 진행률
            is_home[i],                      # 8: 홈 여부
            is_same[i],                      # 9: 선수 연속성
            is_second_half[i],               # 10: 후반전
            # [V6 Phase 1] 새로운 공간 피처 (11-15)
            dist_to_goal[i],                 # 11: 골대 거리
            angle_sin[i],                    # 12: 골대 방향 sin
            angle_cos[i],                    # 13: 골대 방향 cos
            dist_to_sideline[i],             # 14: 터치라인 거리
            dist_to_endline[i],              # 15: 골라인 거리
        ]
        
        # Combine: 16 Scalar + 3 Zone + 16 Type + 9 Result = 44차원
        feat_vec = np.concatenate([scalars, zone_onehot[i], types_onehot[i], results_onehot[i]])
        
        # (1) Start Node
        features.append(feat_vec)
        
        # (2) End Node (마지막 제외)
        if i < n - 1:
            # End 좌표 계산
            ex_real = ex[i] * 105.0
            ey_real = ey[i] * 68.0
            
            # End 위치에서의 공간 피처 재계산
            end_dist_to_goal = np.sqrt((ex_real - GOAL_X)**2 + (ey_real - GOAL_Y)**2) / 105.0
            end_dx_to_goal = GOAL_X - ex_real
            end_dy_to_goal = GOAL_Y - ey_real
            end_angle = np.arctan2(end_dy_to_goal, end_dx_to_goal)
            end_dist_to_sideline = min(ey_real, 68.0 - ey_real) / 68.0
            end_dist_to_endline = min(ex_real, 105.0 - ex_real) / 105.0
            
            end_zone = get_zone(ex[i])
            end_zone_onehot = np.zeros(3, dtype=np.float32)
            end_zone_onehot[end_zone] = 1.0
            
            scalars_end = scalars.copy()
            scalars_end[0] = ex[i]  # End X
            scalars_end[1] = ey[i]  # End Y
            scalars_end[2] = cumsum_dx[i]  # 누적 흐름 업데이트
            scalars_end[3] = cumsum_dy[i]
            # End 위치의 공간 피처
            scalars_end[11] = end_dist_to_goal
            scalars_end[12] = np.sin(end_angle)
            scalars_end[13] = np.cos(end_angle)
            scalars_end[14] = end_dist_to_sideline
            scalars_end[15] = end_dist_to_endline
            
            feat_vec_end = np.concatenate([scalars_end, end_zone_onehot, types_onehot[i], results_onehot[i]])
            features.append(feat_vec_end)
            
    return np.array(features, dtype=np.float32)

# 입력 차원 자동 계산
# Scalar(16) + Zone(3) + Type(16) + Result(9) = 44
dummy_group = train_df.iloc[:5].copy()
dummy_feat = make_features(dummy_group)
INPUT_DIM = dummy_feat.shape[1]

print(f"✅ [V6 Phase 1] Spatial Features Added!")
print(f"   Input Dimension: {INPUT_DIM}")
print(f"   - Base Scalars: 11")
print(f"   - Spatial Features: 5 (goal_dist, angle_sin/cos, sideline_dist, endline_dist)")
print(f"   - Field Zones: 3 (defensive/midfield/attacking)")
print(f"   - Event Types: 16")
print(f"   - Results: 9")
print(f"   Total: 16 + 3 + 16 + 9 = {INPUT_DIM}")

✅ [V6 Phase 1] Spatial Features Added!
   Input Dimension: 44
   - Base Scalars: 11
   - Spatial Features: 5 (goal_dist, angle_sin/cos, sideline_dist, endline_dist)
   - Field Zones: 3 (defensive/midfield/attacking)
   - Event Types: 16
   - Results: 9
   Total: 16 + 3 + 16 + 9 = 44


In [ ]:
class SoccerDataset(Dataset):
    def __init__(self, df, mode='train'):
        self.mode = mode
        self.episodes = []
        self.targets = []
        self.team_ids = []
        self.episode_ids = []
        
        # 순서 섞임 방지
        grouped = df.groupby(ID_COL, sort=False)
        
        for name, group in tqdm(grouped, desc=f"Dataset ({mode})"):
            if mode == 'train' and len(group) < 2: continue
            
            seq = make_features(group)

            team_id = group.iloc[0]['team_id']
            
            if mode == 'train':
                last = group.iloc[-1]
                self.targets.append([last['end_x']/105.0, last['end_y']/68.0])
                self.episodes.append(seq)
                self.team_ids.append(team_id)
            else:
                self.episodes.append(seq)
                self.team_ids.append(team_id)
                self.episode_ids.append(str(name))

    def __len__(self): return len(self.episodes)
    
    def __getitem__(self, idx):
        seq = torch.FloatTensor(self.episodes[idx])
        if len(seq) > MAX_SEQ_LEN: seq = seq[-MAX_SEQ_LEN:]
        
        if self.mode == 'train':
            # Train 반환: (seq, target, team_id)
            return seq, torch.FloatTensor(self.targets[idx]), self.team_ids[idx]
        else:
            # Test 반환: (seq, team_id, episode_id)
            return seq, self.team_ids[idx], self.episode_ids[idx]

# [V7 수정] collate_fn: lengths 정보 추가
def collate_fn(batch):
    seqs = [b[0] for b in batch]
    lens = torch.LongTensor([len(s) for s in seqs])
    padded = pad_sequence(seqs, batch_first=True, padding_value=0)
    mask = torch.arange(padded.size(1))[None, :] >= lens[:, None]
    
    # 배치 첫 번째 요소로 모드를 판단
    elem = batch[0]
    
    # Test Mode 확인 (3번째 요소가 문자열 ID인 경우)
    if isinstance(elem[2], str):
        # Test: b[1]이 Team ID
        team_ids = torch.LongTensor([b[1] for b in batch])
        episode_ids = [b[2] for b in batch]
        return padded, mask, lens, team_ids, episode_ids
        
    else:
        # Train Mode: b[1]은 Target(좌표), b[2]가 Team ID
        targets = torch.stack([b[1] for b in batch])
        team_ids_train = torch.LongTensor([b[2] for b in batch]) 
        return padded, targets, mask, lens, team_ids_train

# 데이터셋 생성
full_dataset = SoccerDataset(train_df, mode='train')
test_dataset = SoccerDataset(test_df, mode='test')

# 에피소드 길이 분포 분석
episode_lengths = [len(ep) for ep in full_dataset.episodes]
print(f"\n📊 [V7] 에피소드 길이 분포 분석")
print(f"   총 에피소드: {len(episode_lengths):,}개")
print(f"   평균 길이: {np.mean(episode_lengths):.2f}")
print(f"   중앙값: {np.median(episode_lengths):.2f}")
print(f"   짧은 에피소드 (<= {SHORT_THRESHOLD}): {sum(1 for l in episode_lengths if l <= SHORT_THRESHOLD):,}개 ({100*sum(1 for l in episode_lengths if l <= SHORT_THRESHOLD)/len(episode_lengths):.1f}%)")
print(f"   중간 에피소드 ({SHORT_THRESHOLD} ~ {LONG_THRESHOLD}): {sum(1 for l in episode_lengths if SHORT_THRESHOLD < l <= LONG_THRESHOLD):,}개 ({100*sum(1 for l in episode_lengths if SHORT_THRESHOLD < l <= LONG_THRESHOLD)/len(episode_lengths):.1f}%)")
print(f"   긴 에피소드 (> {LONG_THRESHOLD}): {sum(1 for l in episode_lengths if l > LONG_THRESHOLD):,}개 ({100*sum(1 for l in episode_lengths if l > LONG_THRESHOLD)/len(episode_lengths):.1f}%)")

In [ ]:
# ======================================================
# [V7] 길이별 앙상블: LSTM + Transformer 모델 정의
# ======================================================

# ===== 1. Sinusoidal Positional Encoding =====
class SinusoidalPositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=100):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

# ===== 2. Transformer 모델 (긴 에피소드용) =====
class TransformerPredictor(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, dropout, max_seq_len=MAX_SEQ_LEN):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        self.pos_encoder = SinusoidalPositionalEncoding(d_model, max_len=max_seq_len)
        self.dropout = nn.Dropout(dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model*4,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, 2)
        )

    def forward(self, x, mask=None):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = self.dropout(x)
        output = self.transformer(x, src_key_padding_mask=mask)

        # Valid Token Mean Pooling
        valid_mask = (~mask).unsqueeze(-1).float()
        mean_out = (output * valid_mask).sum(dim=1) / valid_mask.sum(dim=1).clamp(min=1e-9)
        return self.fc(mean_out)

# ===== 3. LSTM 모델 (짧은 에피소드용) =====
class LSTMPredictor(nn.Module):
    """
    Bi-LSTM + Attention 모델
    - 짧은 시퀀스에 최적화
    - 순차적 정보 보존
    - Transformer보다 적은 파라미터
    """
    def __init__(self, input_dim, hidden_dim, num_layers, dropout, bidirectional=True):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        # LSTM Layer
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional
        )
        
        # Attention Mechanism
        lstm_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        self.attention = nn.Sequential(
            nn.Linear(lstm_output_dim, lstm_output_dim // 2),
            nn.Tanh(),
            nn.Linear(lstm_output_dim // 2, 1)
        )
        
        # Final Prediction Layer
        self.fc = nn.Sequential(
            nn.Linear(lstm_output_dim, lstm_output_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(lstm_output_dim // 2, 2)
        )
    
    def forward(self, x, mask=None, lengths=None):
        """
        Args:
            x: (batch, seq_len, input_dim)
            mask: (batch, seq_len) - True for padding
            lengths: (batch,) - 실제 시퀀스 길이
        """
        batch_size = x.size(0)
        seq_len = x.size(1)
        
        # Pack sequence for efficient LSTM processing
        if lengths is not None:
            # lengths를 CPU로 옮기고 sort
            lengths_cpu = lengths.cpu()
            packed_x = pack_padded_sequence(x, lengths_cpu, batch_first=True, enforce_sorted=False)
            packed_output, (hidden, cell) = self.lstm(packed_x)
            # [FIX] total_length를 지정하여 원래 길이로 복원
            lstm_out, _ = pad_packed_sequence(packed_output, batch_first=True, total_length=seq_len)
        else:
            lstm_out, (hidden, cell) = self.lstm(x)
        
        # Attention Weights 계산
        # lstm_out: (batch, seq_len, hidden*2)
        attn_weights = self.attention(lstm_out)  # (batch, seq_len, 1)
        
        # Mask 적용 (패딩된 부분은 -inf로)
        if mask is not None:
            attn_weights = attn_weights.masked_fill(mask.unsqueeze(-1), float('-inf'))
        
        attn_weights = torch.softmax(attn_weights, dim=1)  # (batch, seq_len, 1)
        
        # Weighted Sum
        context = torch.sum(lstm_out * attn_weights, dim=1)  # (batch, hidden*2)
        
        # Final Prediction
        return self.fc(context)

# ===== 4. 모델 초기화 =====
transformer_model = TransformerPredictor(INPUT_DIM, D_MODEL, NHEAD, NUM_LAYERS, DROPOUT).to(DEVICE)
lstm_model = LSTMPredictor(INPUT_DIM, HIDDEN_DIM, LSTM_LAYERS, DROPOUT, BIDIRECTIONAL).to(DEVICE)

print("=" * 60)
print("✅ [V7] 길이별 앙상블 모델 준비 완료")
print("=" * 60)
print(f"📊 Transformer 모델 (긴 에피소드 > {LONG_THRESHOLD})")
print(f"   - Input Dim: {INPUT_DIM}")
print(f"   - D_Model: {D_MODEL}")
print(f"   - Heads: {NHEAD}")
print(f"   - Layers: {NUM_LAYERS}")
print(f"   - Parameters: {sum(p.numel() for p in transformer_model.parameters()):,}")
print()
print(f"📊 LSTM 모델 (짧은 에피소드 <= {SHORT_THRESHOLD})")
print(f"   - Input Dim: {INPUT_DIM}")
print(f"   - Hidden Dim: {HIDDEN_DIM}")
print(f"   - Layers: {LSTM_LAYERS}")
print(f"   - Bidirectional: {BIDIRECTIONAL}")
print(f"   - Parameters: {sum(p.numel() for p in lstm_model.parameters()):,}")
print("=" * 60)

In [ ]:
# ======================================================
# [V7] 길이별 앙상블: LSTM + Transformer 학습
# ======================================================
from torch.optim.lr_scheduler import CosineAnnealingLR

SEEDS = [42, 2024, 777]
print(f"\n{'='*70}")
print(f"🚀 [V7] 길이별 앙상블 학습 시작")
print(f"{'='*70}")
print(f"전략: LSTM과 Transformer를 각각 학습하여 추론 시 길이별로 선택")
print(f"  - LSTM: 짧은 시퀀스에 최적화 (파라미터 효율적)")
print(f"  - Transformer: 긴 시퀀스에 최적화 (long-range dependency)")
print(f"{'='*70}\n")

# 히스토리 저장용
lstm_histories = []
transformer_histories = []

for i, seed in enumerate(SEEDS):
    print(f"\n{'='*70}")
    print(f"📦 [모델 세트 {i+1}/3] Seed {seed} 학습 시작")
    print(f"{'='*70}")
    seed_everything(seed)
    
    # 데이터셋 분할
    train_idx, val_idx = train_test_split(range(len(full_dataset)), test_size=0.2, random_state=seed)
    train_loader = DataLoader(torch.utils.data.Subset(full_dataset, train_idx), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(torch.utils.data.Subset(full_dataset, val_idx), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    # ===== 1. LSTM 모델 학습 =====
    print(f"\n🔹 [1/2] LSTM 모델 학습 (짧은 에피소드 특화)")
    print(f"   목표: 짧은 시퀀스에서 최고 성능 달성")
    
    lstm = LSTMPredictor(INPUT_DIM, HIDDEN_DIM, LSTM_LAYERS, DROPOUT, BIDIRECTIONAL).to(DEVICE)
    lstm_optimizer = optim.AdamW(lstm.parameters(), lr=LR_BASE, weight_decay=1e-4)
    lstm_scheduler = CosineAnnealingLR(lstm_optimizer, T_max=EPOCHS_BASE - WARMUP_EPOCHS, eta_min=1e-6)
    criterion = nn.HuberLoss(delta=1.0)
    
    best_lstm_dist = float('inf')
    lstm_history = {'train_loss': [], 'val_dist': [], 'lr': []}
    
    for epoch in range(EPOCHS_BASE):
        # Warmup
        if epoch < WARMUP_EPOCHS:
            lr = LR_BASE * (epoch + 1) / WARMUP_EPOCHS
            for param_group in lstm_optimizer.param_groups:
                param_group['lr'] = lr
        
        # Training
        lstm.train()
        train_loss = 0.0
        for seqs, targets, mask, lens, _ in train_loader:
            seqs, targets, mask, lens = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE), lens.to(DEVICE)
            
            lstm_optimizer.zero_grad()
            pred = lstm(seqs, mask, lens)
            loss = criterion(pred, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(lstm.parameters(), GRAD_CLIP)
            lstm_optimizer.step()
            train_loss += loss.item()
        
        # Validation
        lstm.eval()
        dists = []
        with torch.no_grad():
            for seqs, targets, mask, lens, _ in val_loader:
                seqs, targets, mask, lens = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE), lens.to(DEVICE)
                pred = lstm(seqs, mask, lens)
                p_real = pred.cpu().numpy() * np.array([105.0, 68.0])
                t_real = targets.cpu().numpy() * np.array([105.0, 68.0])
                dists.extend(np.sqrt(np.sum((p_real - t_real)**2, axis=1)))
        
        avg_dist = np.mean(dists)
        avg_train_loss = train_loss / len(train_loader)
        
        if epoch >= WARMUP_EPOCHS:
            lstm_scheduler.step()
        
        current_lr = lstm_optimizer.param_groups[0]['lr']
        lstm_history['train_loss'].append(avg_train_loss)
        lstm_history['val_dist'].append(avg_dist)
        lstm_history['lr'].append(current_lr)
        
        if (epoch + 1) % 10 == 0 or epoch < 3:
            print(f"   [Epoch {epoch+1:2d}/{EPOCHS_BASE}] Loss: {avg_train_loss:.4f} | Val Dist: {avg_dist:.4f}m | LR: {current_lr:.6f}")
        
        if avg_dist < best_lstm_dist:
            best_lstm_dist = avg_dist
            torch.save(lstm.state_dict(), f'v7_lstm_{i}.pth')
    
    lstm_history['best_dist'] = best_lstm_dist
    lstm_history['seed'] = seed
    lstm_histories.append(lstm_history)
    print(f"   ✅ LSTM 완료. Best Val Dist: {best_lstm_dist:.4f}m")
    
    # ===== 2. Transformer 모델 학습 =====
    print(f"\n🔹 [2/2] Transformer 모델 학습 (긴 에피소드 특화)")
    print(f"   목표: 긴 시퀀스에서 최고 성능 달성")
    
    transformer = TransformerPredictor(INPUT_DIM, D_MODEL, NHEAD, NUM_LAYERS, DROPOUT).to(DEVICE)
    transformer_optimizer = optim.AdamW(transformer.parameters(), lr=LR_BASE, weight_decay=1e-4)
    transformer_scheduler = CosineAnnealingLR(transformer_optimizer, T_max=EPOCHS_BASE - WARMUP_EPOCHS, eta_min=1e-6)
    
    best_transformer_dist = float('inf')
    transformer_history = {'train_loss': [], 'val_dist': [], 'lr': []}
    
    for epoch in range(EPOCHS_BASE):
        # Warmup
        if epoch < WARMUP_EPOCHS:
            lr = LR_BASE * (epoch + 1) / WARMUP_EPOCHS
            for param_group in transformer_optimizer.param_groups:
                param_group['lr'] = lr
        
        # Training
        transformer.train()
        train_loss = 0.0
        for seqs, targets, mask, lens, _ in train_loader:
            seqs, targets, mask = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE)
            
            transformer_optimizer.zero_grad()
            pred = transformer(seqs, mask)
            loss = criterion(pred, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(transformer.parameters(), GRAD_CLIP)
            transformer_optimizer.step()
            train_loss += loss.item()
        
        # Validation
        transformer.eval()
        dists = []
        with torch.no_grad():
            for seqs, targets, mask, lens, _ in val_loader:
                seqs, targets, mask = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE)
                pred = transformer(seqs, mask)
                p_real = pred.cpu().numpy() * np.array([105.0, 68.0])
                t_real = targets.cpu().numpy() * np.array([105.0, 68.0])
                dists.extend(np.sqrt(np.sum((p_real - t_real)**2, axis=1)))
        
        avg_dist = np.mean(dists)
        avg_train_loss = train_loss / len(train_loader)
        
        if epoch >= WARMUP_EPOCHS:
            transformer_scheduler.step()
        
        current_lr = transformer_optimizer.param_groups[0]['lr']
        transformer_history['train_loss'].append(avg_train_loss)
        transformer_history['val_dist'].append(avg_dist)
        transformer_history['lr'].append(current_lr)
        
        if (epoch + 1) % 10 == 0 or epoch < 3:
            print(f"   [Epoch {epoch+1:2d}/{EPOCHS_BASE}] Loss: {avg_train_loss:.4f} | Val Dist: {avg_dist:.4f}m | LR: {current_lr:.6f}")
        
        if avg_dist < best_transformer_dist:
            best_transformer_dist = avg_dist
            torch.save(transformer.state_dict(), f'v7_transformer_{i}.pth')
    
    transformer_history['best_dist'] = best_transformer_dist
    transformer_history['seed'] = seed
    transformer_histories.append(transformer_history)
    print(f"   ✅ Transformer 완료. Best Val Dist: {best_transformer_dist:.4f}m")
    
    print(f"\n📊 [모델 세트 {i+1}] 요약")
    print(f"   LSTM Best:        {best_lstm_dist:.4f}m")
    print(f"   Transformer Best: {best_transformer_dist:.4f}m")

print(f"\n{'='*70}")
print(f"✅ 전체 학습 완료!")
print(f"{'='*70}")
print(f"📊 LSTM 평균 Best Distance:        {np.mean([h['best_dist'] for h in lstm_histories]):.4f}m")
print(f"📊 Transformer 평균 Best Distance: {np.mean([h['best_dist'] for h in transformer_histories]):.4f}m")
print(f"{'='*70}")

In [ ]:
# ======================================================
# [V7] 길이별 동적 앙상블 추론
# ======================================================
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print(f"\n{'='*70}")
print(f"🔮 [V7] 길이별 동적 앙상블 추론 시작")
print(f"{'='*70}")

# ===== 1. 모델 로드 (각 3개씩) =====
lstm_models = []
transformer_models = []

for i in range(3):
    # LSTM 모델
    lstm = LSTMPredictor(INPUT_DIM, HIDDEN_DIM, LSTM_LAYERS, DROPOUT, BIDIRECTIONAL).to(DEVICE)
    lstm.load_state_dict(torch.load(f'v7_lstm_{i}.pth'))
    lstm.eval()
    lstm_models.append(lstm)
    
    # Transformer 모델
    transformer = TransformerPredictor(INPUT_DIM, D_MODEL, NHEAD, NUM_LAYERS, DROPOUT).to(DEVICE)
    transformer.load_state_dict(torch.load(f'v7_transformer_{i}.pth'))
    transformer.eval()
    transformer_models.append(transformer)

print(f"✅ 모델 로드 완료")
print(f"   - LSTM 모델: 3개")
print(f"   - Transformer 모델: 3개")

# ===== 2. 길이별 앙상블 가중치 =====
seed_weights = [0.25, 0.25, 0.5]  # 3개 모델 앙상블 가중치

def length_based_ensemble(lstm_pred, transformer_pred, length):
    """
    길이에 따라 동적으로 앙상블 가중치 적용
    
    Args:
        lstm_pred: LSTM 예측 (batch, 2)
        transformer_pred: Transformer 예측 (batch, 2)
        length: 에피소드 길이 (스칼라)
    
    Returns:
        앙상블된 예측 (batch, 2)
    """
    if length <= SHORT_THRESHOLD:
        # 짧은 에피소드: LSTM 100%
        return lstm_pred
    elif length <= LONG_THRESHOLD:
        # 중간 에피소드: LSTM 60% + Transformer 40%
        return 0.6 * lstm_pred + 0.4 * transformer_pred
    else:
        # 긴 에피소드: Transformer 100%
        return transformer_pred

# ===== 3. 추론 =====
results = []
length_distribution = {'short': 0, 'medium': 0, 'long': 0}

print(f"\n🔄 추론 중...")
with torch.no_grad():
    for seqs, mask, lens, team_ids, episode_ids in tqdm(test_loader, desc="Inference"):
        seqs, mask, lens = seqs.to(DEVICE), mask.to(DEVICE), lens.to(DEVICE)
        batch_size = seqs.size(0)
        
        # 각 배치 내 샘플별로 처리
        for b in range(batch_size):
            seq = seqs[b:b+1]
            m = mask[b:b+1]
            l = lens[b:b+1]
            length = lens[b].item()
            eid = episode_ids[b]
            
            # 길이 분포 기록
            if length <= SHORT_THRESHOLD:
                length_distribution['short'] += 1
            elif length <= LONG_THRESHOLD:
                length_distribution['medium'] += 1
            else:
                length_distribution['long'] += 1
            
            # 3개 모델에서 각각 예측
            lstm_preds = []
            transformer_preds = []
            
            for j in range(3):
                lstm_out = lstm_models[j](seq, m, l).cpu().numpy()
                transformer_out = transformer_models[j](seq, m).cpu().numpy()
                lstm_preds.append(lstm_out)
                transformer_preds.append(transformer_out)
            
            # Seed 앙상블 (3개 모델)
            lstm_ensemble = (lstm_preds[0] * seed_weights[0] + 
                           lstm_preds[1] * seed_weights[1] + 
                           lstm_preds[2] * seed_weights[2])
            transformer_ensemble = (transformer_preds[0] * seed_weights[0] + 
                                  transformer_preds[1] * seed_weights[1] + 
                                  transformer_preds[2] * seed_weights[2])
            
            # 길이별 앙상블
            final_pred = length_based_ensemble(lstm_ensemble, transformer_ensemble, length)
            
            # 좌표 변환
            px = final_pred[0, 0] * 105.0
            py = final_pred[0, 1] * 68.0
            
            results.append({'game_episode': eid, 'pred_x': px, 'pred_y': py})

# ===== 4. 결과 통계 =====
total = sum(length_distribution.values())
print(f"\n📊 테스트 데이터 길이 분포:")
print(f"   짧은 에피소드 (<= {SHORT_THRESHOLD}):  {length_distribution['short']:5d}개 ({100*length_distribution['short']/total:.1f}%) → LSTM 100%")
print(f"   중간 에피소드 ({SHORT_THRESHOLD}~{LONG_THRESHOLD}): {length_distribution['medium']:5d}개 ({100*length_distribution['medium']/total:.1f}%) → LSTM 60% + Transformer 40%")
print(f"   긴 에피소드 (> {LONG_THRESHOLD}):   {length_distribution['long']:5d}개 ({100*length_distribution['long']/total:.1f}%) → Transformer 100%")

# ===== 5. 제출 파일 생성 =====
pred_df = pd.DataFrame(results)

SUBMISSION_PATH = "open_track1/sample_submission.csv"
if os.path.exists(SUBMISSION_PATH):
    sub = pd.read_csv(SUBMISSION_PATH)
else:
    sub = pd.read_csv(TEST_META_PATH)
    col_map = {'episode_id': 'game_episode'}
    sub = sub.rename(columns=col_map)
    sub = sub[['game_episode']]

print(f"\n📄 제출 파일 생성 중...")
print(f"   Submission Shape: {sub.shape}, Prediction Shape: {pred_df.shape}")

final_sub = pd.merge(sub[['game_episode']], pred_df, on='game_episode', how='left')
final_sub = final_sub.rename(columns={'pred_x': 'end_x', 'pred_y': 'end_y'})

nan_count = final_sub.isnull().sum().sum()
if nan_count > 0:
    print(f"   ⚠️ 경고: {nan_count}개의 결측치가 발생했습니다.")
    final_sub = final_sub.fillna(50.0)

filename = "v7_length_ensemble_submit.csv"
final_sub.to_csv(filename, index=False)

print(f"\n{'='*70}")
print(f"✅ [V7] 길이별 동적 앙상블 추론 완료!")
print(f"{'='*70}")
print(f"📁 제출 파일: {filename}")
print(f"📊 전략 요약:")
print(f"   - 짧은 에피소드: LSTM의 순차적 정보 처리 능력 활용")
print(f"   - 중간 에피소드: 두 모델의 강점을 균형있게 결합")
print(f"   - 긴 에피소드: Transformer의 long-range dependency 포착 능력 활용")
print(f"{'='*70}")
print(final_sub.head())

In [ ]:
# ======================================================
# [V7] 길이별 앙상블 학습 곡선 시각화
# ======================================================
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
# plt.rcParams['font.family'] = 'NanumGothic'  # Linux
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

fig.suptitle('[V7] 길이별 앙상블: LSTM + Transformer 학습 곡선', fontsize=18, fontweight='bold')

colors_lstm = ['#1f77b4', '#ff7f0e', '#2ca02c']
colors_transformer = ['#d62728', '#9467bd', '#8c564b']

# ===== 1. LSTM Training Loss =====
ax1 = fig.add_subplot(gs[0, 0])
for i, history in enumerate(lstm_histories):
    epochs = range(1, len(history['train_loss']) + 1)
    ax1.plot(epochs, history['train_loss'], label=f'Seed {history["seed"]}', 
             color=colors_lstm[i], linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Huber Loss')
ax1.set_title('LSTM Training Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# ===== 2. LSTM Validation Distance =====
ax2 = fig.add_subplot(gs[0, 1])
for i, history in enumerate(lstm_histories):
    epochs = range(1, len(history['val_dist']) + 1)
    ax2.plot(epochs, history['val_dist'], label=f'Seed {history["seed"]}', 
             color=colors_lstm[i], linewidth=2)
    best_idx = np.argmin(history['val_dist'])
    best_val = history['val_dist'][best_idx]
    ax2.scatter(best_idx + 1, best_val, color=colors_lstm[i], s=100, zorder=5, marker='*')
    ax2.annotate(f'{best_val:.2f}m', xy=(best_idx + 1, best_val), 
                 xytext=(5, 5), textcoords='offset points', fontsize=9, color=colors_lstm[i])
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Distance (m)')
ax2.set_title('LSTM Validation Distance')
ax2.legend()
ax2.grid(True, alpha=0.3)

# ===== 3. LSTM Best Distances =====
ax3 = fig.add_subplot(gs[0, 2])
best_dists_lstm = [h['best_dist'] for h in lstm_histories]
seeds_lstm = [h['seed'] for h in lstm_histories]
bars = ax3.bar(range(len(best_dists_lstm)), best_dists_lstm, color=colors_lstm)
ax3.set_xticks(range(len(best_dists_lstm)))
ax3.set_xticklabels([f'Seed\n{s}' for s in seeds_lstm])
ax3.set_ylabel('Best Validation Distance (m)')
ax3.set_title('LSTM Best Performance')
ax3.grid(True, alpha=0.3, axis='y')
for i, (bar, dist) in enumerate(zip(bars, best_dists_lstm)):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height, f'{dist:.4f}m',
             ha='center', va='bottom', fontweight='bold', fontsize=9)

# ===== 4. Transformer Training Loss =====
ax4 = fig.add_subplot(gs[1, 0])
for i, history in enumerate(transformer_histories):
    epochs = range(1, len(history['train_loss']) + 1)
    ax4.plot(epochs, history['train_loss'], label=f'Seed {history["seed"]}', 
             color=colors_transformer[i], linewidth=2)
ax4.set_xlabel('Epoch')
ax4.set_ylabel('Huber Loss')
ax4.set_title('Transformer Training Loss')
ax4.legend()
ax4.grid(True, alpha=0.3)

# ===== 5. Transformer Validation Distance =====
ax5 = fig.add_subplot(gs[1, 1])
for i, history in enumerate(transformer_histories):
    epochs = range(1, len(history['val_dist']) + 1)
    ax5.plot(epochs, history['val_dist'], label=f'Seed {history["seed"]}', 
             color=colors_transformer[i], linewidth=2)
    best_idx = np.argmin(history['val_dist'])
    best_val = history['val_dist'][best_idx]
    ax5.scatter(best_idx + 1, best_val, color=colors_transformer[i], s=100, zorder=5, marker='*')
    ax5.annotate(f'{best_val:.2f}m', xy=(best_idx + 1, best_val), 
                 xytext=(5, 5), textcoords='offset points', fontsize=9, color=colors_transformer[i])
ax5.set_xlabel('Epoch')
ax5.set_ylabel('Distance (m)')
ax5.set_title('Transformer Validation Distance')
ax5.legend()
ax5.grid(True, alpha=0.3)

# ===== 6. Transformer Best Distances =====
ax6 = fig.add_subplot(gs[1, 2])
best_dists_transformer = [h['best_dist'] for h in transformer_histories]
seeds_transformer = [h['seed'] for h in transformer_histories]
bars = ax6.bar(range(len(best_dists_transformer)), best_dists_transformer, color=colors_transformer)
ax6.set_xticks(range(len(best_dists_transformer)))
ax6.set_xticklabels([f'Seed\n{s}' for s in seeds_transformer])
ax6.set_ylabel('Best Validation Distance (m)')
ax6.set_title('Transformer Best Performance')
ax6.grid(True, alpha=0.3, axis='y')
for i, (bar, dist) in enumerate(zip(bars, best_dists_transformer)):
    height = bar.get_height()
    ax6.text(bar.get_x() + bar.get_width()/2., height, f'{dist:.4f}m',
             ha='center', va='bottom', fontweight='bold', fontsize=9)

# ===== 7. Learning Rate Schedule (공통) =====
ax7 = fig.add_subplot(gs[2, 0])
history = lstm_histories[0]  # LR은 모두 동일
epochs = range(1, len(history['lr']) + 1)
ax7.plot(epochs, history['lr'], color='purple', linewidth=2)
ax7.axvline(x=WARMUP_EPOCHS, color='red', linestyle='--', label=f'Warmup End (Epoch {WARMUP_EPOCHS})')
ax7.set_xlabel('Epoch')
ax7.set_ylabel('Learning Rate')
ax7.set_title('Learning Rate Schedule (공통)')
ax7.legend()
ax7.grid(True, alpha=0.3)
ax7.set_yscale('log')

# ===== 8. LSTM vs Transformer 비교 =====
ax8 = fig.add_subplot(gs[2, 1])
avg_lstm = np.mean(best_dists_lstm)
avg_transformer = np.mean(best_dists_transformer)
models = ['LSTM\n(짧은 에피소드)', 'Transformer\n(긴 에피소드)']
avgs = [avg_lstm, avg_transformer]
colors_comp = ['#1f77b4', '#d62728']
bars = ax8.bar(models, avgs, color=colors_comp, alpha=0.7, edgecolor='black', linewidth=2)
ax8.set_ylabel('평균 Best Distance (m)')
ax8.set_title('LSTM vs Transformer 성능 비교')
ax8.grid(True, alpha=0.3, axis='y')
for bar, avg in zip(bars, avgs):
    height = bar.get_height()
    ax8.text(bar.get_x() + bar.get_width()/2., height, f'{avg:.4f}m',
             ha='center', va='bottom', fontweight='bold', fontsize=12)

# ===== 9. 전략 요약 텍스트 =====
ax9 = fig.add_subplot(gs[2, 2])
ax9.axis('off')
summary_text = f"""[V7] 길이별 앙상블 전략

모델 성능:
  LSTM 평균: {avg_lstm:.4f}m
  Transformer 평균: {avg_transformer:.4f}m

추론 전략:
  짧은 (<= {SHORT_THRESHOLD}): LSTM 100%
  중간 ({SHORT_THRESHOLD}~{LONG_THRESHOLD}): 0.6*LSTM + 0.4*Transformer
  긴 (> {LONG_THRESHOLD}): Transformer 100%

이론적 근거:
  LSTM: 짧은 시퀀스에 효율적
        순차 정보 보존
        적은 파라미터
  
  Transformer: 긴 시퀀스에 효율적
               Long-range dependency
               병렬 처리 가능
"""
# [FIX] 명시적으로 폰트 지정 (제목처럼!)
ax9.text(0.05, 0.5, summary_text, fontsize=10, verticalalignment='center',
         fontfamily='Malgun Gothic',  # ← 이게 핵심!
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

plt.savefig('v7_training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n학습 곡선 저장 완료: v7_training_history.png")
print(f"\n최종 결과:")
print(f"{'='*60}")
print(f"LSTM 모델:")
for i, h in enumerate(lstm_histories):
    print(f"  Seed {h['seed']:4d}: {h['best_dist']:.4f}m")
print(f"  평균: {avg_lstm:.4f}m")
print(f"\nTransformer 모델:")
for i, h in enumerate(transformer_histories):
    print(f"  Seed {h['seed']:4d}: {h['best_dist']:.4f}m")
print(f"  평균: {avg_transformer:.4f}m")
print(f"{'='*60}")

# 📊 [V7] K-League 패스 예측 - 길이별 앙상블 전략

## 🎯 핵심 아이디어

이론적 분석에 기반하여, **에피소드 길이에 따라 최적의 모델을 동적으로 선택**하는 앙상블 전략을 구현했습니다.

### 📌 문제 인식

- **평균의 함정**: 평균 에피소드 길이는 23.11개이지만, 이는 대표값이 아님
- **Right-skewed 분포**: 대부분의 에피소드는 중앙값(18개)보다 짧음
- **Transformer의 한계**: 긴 시퀀스에 최적화되어 짧은 시퀀스에서는 비효율적

### 🔬 이론적 근거

| 에피소드 길이 | 특성 | 적합한 모델 | 이유 |
|---|---|---|---|
| **짧음 (≤15)** | 빠른 역습, 단순 패턴 | **LSTM** | 순차 정보 보존, 파라미터 효율적 |
| **중간 (15~40)** | 일반적 공격 전개 | **앙상블** | 두 모델의 강점 결합 |
| **긴 (>40)** | 복잡한 빌드업 | **Transformer** | Long-range dependency 포착 |

## 🏗️ 모델 아키텍처

### 1. LSTM 모델 (짧은 에피소드 특화)
- **Bi-LSTM**: 양방향으로 시퀀스 정보 처리
- **Attention**: 중요한 액션에 가중치 부여
- **장점**: 
  - 작은 파라미터로 효율적 학습
  - 순차적 패턴 보존
  - Overfitting 위험 낮음

### 2. Transformer 모델 (긴 에피소드 특화)
- **Sinusoidal PE**: 위치 정보 인코딩
- **GELU Activation**: 부드러운 비선형성
- **장점**:
  - Long-range dependency 포착
  - 병렬 처리로 빠른 학습
  - 복잡한 패턴 학습 가능

## 🎲 추론 전략

```python
IF episode_length <= 15:
    prediction = LSTM_model(episode)
    weight = 1.0
    
ELIF episode_length <= 40:
    lstm_pred = LSTM_model(episode)
    transformer_pred = Transformer_model(episode)
    prediction = 0.6 * lstm_pred + 0.4 * transformer_pred
    
ELSE:  # > 40
    prediction = Transformer_model(episode)
    weight = 1.0
```

## 📈 기대 효과

1. **대부분의 에피소드(짧은 구간)에서 LSTM 우세** → 전체 성능 향상
2. **각 모델을 강점 구간에 활용** → 최적 성능 달성
3. **이론에 기반한 설계** → 일반화 성능 우수

## 🔧 학습 설정

- **Seeds**: [42, 2024, 777] (3개 앙상블)
- **Epochs**: 40 (Warmup 3 epochs)
- **Optimizer**: AdamW (weight_decay=1e-4)
- **Scheduler**: Cosine Annealing
- **Loss**: Huber Loss (이상치에 강함)
- **Gradient Clipping**: 1.0

## 📚 참고 문헌

이 구현은 다음 이론적 연구에 기반합니다:

- **LSTM vs Transformer 비교 연구** (Machine Learning Mastery, 2024)
- **스포츠 분석에서의 GNN 응용** (arXiv, 2022)
- **축구 이벤트 예측** (KDD 2022, Seq2Event)
- **시계열 예측에서의 모델 선택** (Neural Brain Works, 2025)